In [1]:
import sys
sys.path.append('../')

from src.policy import TwoOptPolicy, RandomPolicy, GreedyPolicy
import numpy as np
import japanize_matplotlib
import matplotlib.pyplot as plt
from pathlib import Path
from tqdm.autonotebook import tqdm
from PIL import Image

plt.style.use('ggplot')

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def visulize_logs(x, logs, path_to_dir: Path, log_step=10):
    for step, (tour_i, score) in enumerate(tqdm(logs)):
        if step % log_step == 0:
            fig, ax = plt.subplots(figsize=(5, 5))
            x_cood = x[tour_i, 0]
            y_cood = x[tour_i, 1]
            x_cood = np.append(x_cood, x_cood[0])
            y_cood = np.append(y_cood, y_cood[0])

            ax.plot(x_cood, y_cood, marker=".", markersize=7, label='city')
            ax.set_title(f"step: {step+1} 総距離: {score:.2f}", fontsize=15)
            ax.legend()
            ax.set_xticks([])
            ax.set_yticks([])

            fig.savefig(
                path_to_dir / f"{step+1}.png",
            )
            plt.close()

def make_gif(paths, save_path, max_step=np.inf):
    imgs = []
    for i, path in enumerate(tqdm(paths)):
        imgs.append(
            Image.open(path)
        )
        if i > max_step:
            break

    imgs[0].save(
        save_path,
        save_all=True,
        append_images=imgs[1:],
        duration=30,
        loop=0,
    )

In [3]:
policy = TwoOptPolicy(
    start_tmp=0.045,
    end_tmp=0.00045,
    timelimit=1.0,
    init_policy=RandomPolicy(),
)

x = np.random.rand(150, 2)
init_tour = policy.init_policy.solve(x[None, :])[0]
best_tour, logs = policy.solve_sample(x, init_tour, return_log=True)
path_to_dir = Path("/workspaces/results/sa_log")
path_to_dir.mkdir(exist_ok=True, parents=True)

In [4]:
visulize_logs(x, logs, path_to_dir, log_step=20)

100%|██████████| 22495/22495 [00:29<00:00, 762.63it/s]


In [6]:
paths = sorted(list(path_to_dir.glob("*.png")), key=lambda x: int(x.stem))
save_path = str("/workspaces/sample.gif")
make_gif(paths, save_path)

100%|██████████| 1212/1212 [00:00<00:00, 7013.68it/s]
